In [17]:
## Издевательство над текстом в ячейках
from IPython.core.display import HTML
HTML("""
<style>

div.text_cell_render h1 { /* Main titles bigger, centered */
font-size: 2em;
line-height:1.1em;
text-align:center;
}

div.text_cell_render h4 { /*  Parts names nearer from text */
margin-bottom: -0.5em;
}

div.text_cell_render table { /*  Parts names nearer from text */
width: auto;
margin-left: 0px;
margin-right: auto;
font-size:1em;
}

div.text_cell_render ul{ /* Customize text cells */
margin-bottom: -0.4em;
columns: 2;
-webkit-columns: 2;
-moz-columns: 2;
list-style-position: inside;
}
div.text_cell_render { /* Customize text cells */
font-size:1.5em;
line-height:1.4em;
padding-left:3em;
padding-right:3em;
}
</style>
""")

In [11]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

In [12]:
def pickle_dump(obj, filepath):
    with open(filepath, 'wb') as file:
        pickle.dump(obj, file)
def pickle_load(filepath):
    with open(filepath, 'rb') as file:
        res = pickle.load(file)
    return res

# Автоматическое извлечение риторических отношений на уровне предложения в русском языке
Захарова Е., БКЛ-142   
Научный руководитель - Толдова С.Ю.

## Информация о дискурсивной структуре может использоваться в задачах

+ извлечения фактов/мнений
+ summarization
+ анализа тональности
+ определения фейковых новостей
+ машинного перевода
+ автоматической генерации текста 

## Теория Риторических Структур
+ формальный анализ дискурсивной структуры текста
+ by Mann, Thompson ... 1980-е
+ изначально предназначалась для задачи автоматической генерации текста
+ сейчас имеет независимый теоретический статус
### Основные идеи:
+ Текст отличается от случайного набора предложений свойствами целостности и связности: каждая из частей текста обладает собственной функцией
+ Части текста на всех уровнях связаны **риторическими отношениями**
+ Текст - древовидная структура

**Элментарная Дискурсивная Единица (ЭДЕ)** - единица дискурса низшего уровня (квант дискурса), обычно совпадает с клаузой

## Риторические отношения 
**Асимметричные**: **Ядро** (Nucleus) - основное значение, **Сателлит** (Satellite) - опциональная часть   
**Симметричные**: 2 и более **Ядер**
#### Асимметричные:
+ Background
+ Cause-Effect
+ Evidence
+ Condition
+ Purpose
+ Concession
+ Preparation
+ Conclusion
+ Elaboration,
+ Antithesis
+ Solutionhood
+ Interpretation-Evaluation
+ Attribution

#### Симметричные:
+ Contrast
+ Restatement
+ Sequence
+ Joint
+ Comparison
+ Same-unit

## Пример риторической структуры
![](dt_example.png)

**Дискурсивные маркеры** - сигнализируют присутсвие того или иного отношения (*если* -> condition и т.д.)

## Автоматический анализ дискурса 
Две подзадачи: сегментация (деление на ЭДЕ), парсинг.  
Для английского: сегментация - state-of-the-art, парсинг - еще нет.  
Для русского - вообще ничего.
#### Существующие парсеры - идеи:
+ Машинное обучение < правила основанные на маркерах (отношения без маркеров, не 100% точные маркеры, неоднозначные маркеры)
+ Признаки - лучше группы маркеров или словрь по н-граммам, чем использование отдельных маркеров
+ Дискриминативный подход: отдельные алгоритмы для уровня предложения и уровня текста (разная структура) 


**Цель** - алгоритм извлечения риторических отношений на уровне предложения в русских письменных текстах (надеюсь, что в будущем станет частью полноценного парсера)

## Данные
Корпус Russian RST Bank (73 текста, ~ по 30 предложений)  
**Объекты** - пары ЭДЕ внутри предложения, в рамках окна ширины 5 (4072 объекта)   
**Целева переменная** - наличие отношений, тип отношений, нуклеарность (какая из ЭДЕ - ядро), всего 26 классов (17 отношений)

## Распределение классов
![](classes.png)

## Признаки (19850)
+ позиция в тексте 
+ расстояние (в ЭДЕ)
+ длина в токенах 
+ стоит в начале предложения 
+ стоит в конце предложения 
+ количество общих слов (по леммам)
+ векторы doc2vec  
+ векторы word2vec первого и последнего токена 
+ наличие маркера риторических отношений (списки маркеров для каждого из отношений, всего Х списков)
+ Count Vectorizer по токенам и би- и триграммам токенов (знаки препинания учитываются, стоп-слова не удаляются, без лемматизации) 
+ Count Vectorizer по POS-тегам и би- и триграммам POS-тегов

Doc2Vec, Word2Vec - обучение на корпусе серебр. стандарт ГИКРЯ, Викиновости (без лемматизации, без удаления стоп-слов, знаки препинания учитывались)

### Настройка гиперпараметров с помощью Grid Search

In [24]:
gs.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1500, n_jobs=3,
            oob_score=False, random_state=669, verbose=0, warm_start=False)

## Oversampling
+ Дисбаланс классов (самый крупный - no_rеlation, значительно больше объектов, чем в остальных)
+ Oversampling c помощью SMOTE (5 соседей) 
+ В итоге - одинаковое число объектов во всех классах (~1800, без дублирования только класс no_realtion)
+ Гипотеза: может влияеть на оценку качества при кросс-валидации => отложенная тестовая выборка, не подвергалась оверсэмплингу

In [ ]:
## здесь будут графики -  обучающая выборка (SMOTE) и тестовая выборка - оригинальные объекты 

## Качество 
### Cross-val (SMOTE)
#### Среднее взвешенное по все классам
| Precision | Recall | F1       |
|-----------|--------|----------|
| 0.97      | 0.97   | 0.97     |
#### No_relation - класс с самым низким качесвтом
| Precision | Recall | F1       |
|-----------|--------|----------|
| 0.78      | 0.51   | 0.61     |
#### Confusion matrix
![](norm_cv.png)

### Test (обучение - SMOTE, тест - только оригинальные объекты)
#### Cреднее взвешенное по всем классам
| Precision | Recall | F1       |
|-----------|--------|----------|
| 0.51      | 0.54   | 0.51     |
#### Confusion matrix
![](norm_test.png)
**Тенденция относить объекты миноритарных классов к крупным классам сохраняется, несмотря на сбалансированность классов в обучающей выборке**

In [5]:
# Display the Dive visualization for this data
from IPython.core.display import display, HTML
data = pickle_load('data.pkl')
features = data.columns

# set the sprite_size based on the number of records in dataset,
# larger datasets can crash the browser if the size is too large (>50000)
sprite_size = 32 if len(data.index)>50000 else 64

jsonstr = data.to_json(orient='records')
# Create Facets template  
HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-dist/facets-jupyter.html">
        <facets-dive sprite-image-width="{sprite_size}" sprite-image-height="{sprite_size}" id="elem" height="650"></facets-dive>
        <script>
          document.querySelector("#elem").data = {jsonstr};
        </script>"""

# Load the json dataset and the sprite_size into the template
html = HTML_TEMPLATE.format(jsonstr=jsonstr, sprite_size=sprite_size)

## Наиболее важные признаки (первые 150)
+ расстояние между ЭДЕ в паре (в ЭДЕ)
+ позиция в тексте (ЭДЕ 1 и 2)
+ наличие маркеров condition, наличие "если" (Count Vectorizer)
+ наличие маркеров concession, наличие "несмотря" (ЭДЕ 1 и 2)
+ наличие маркеров purpose, наличие "чтобы" (ЭДЕ 1 и 2)
+ наличие маркеров contrast (ЭДЕ 1 и 2)
+ наличие маркеров attribution (ЭДЕ 1 и 2)
+ наличие "поскольку" в ЭДЕ 2
+ счетчик PRON в ЭДЕ 2
+ счетчик NOUN в ЭДЕ 2
+ счетчик VERB в ЭДЕ 2
+ счетчик CCONJ в ЭДЕ 2
+ вектор w2v первого слова ЭДЕ 2
+ векоры d2v (ЭДЕ 1 и 2)

In [6]:
# Display the template
display(HTML(html))

## Вывод
### Улучшение алгоритма
+ **больше данных**
+ добавление синтаксических признаков (нужен быстрый парсер)
+ feature_selection
+ расширение списка маркеров/дополнительное разделение по вероятности внутри отношений


### Дальнейшее исследование
+ полноценный парсер с сегментацией
+ не только письменная, но и устная речь (введение новых отношений, другие правила сегментации)

**На данном этапе алгоритм может быть полезен для предварительной разметки => ускорение и облегчение процесса ручной разметки**
